<h1>PROGRAMMING TETRIS<h1>

In [ ]:
# Libraries required
import pygame
import random
import numpy as np
pygame.font.init()

First, let's define the datastructures for our setting.

How do we represent our playground? It has to be a regular grid, so let's use a
multidimensional array in python to represent it

In [ ]:
def create_grid(x, y, locked_positions={}):
    grid = [[(0, 0, 0) for _ in range(x)] for _ in range(y)]

    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (j, i) in locked_positions:
                c = locked_positions[(j, i)]
                grid[i][j] = c
    return grid

This function does exactly what the name says, it creates a grid 
of dimensions x and y and returns it. But what's the variable locked_positions all about?
<i>locked_positions<i> will hold the positions which are "locked" by the presence of a piece in our game. The positions are represented by indices into the grid (j,i) (j - for the column, i - for the row). <i>ocked_positions<i> is a dictionary, where you can look up the colour c corresponding to every (j,i) position in the grid.

In [ ]:
# Let's call our function:

# define x and y
x = 10
y = 20

# let's create an empty dictionary for now, as we have no pieces in our game yet
locked_positions = {}

grid = create_grid(x,y,locked_positions)



Now we created our grid! In order to visualize it, we need to create the rest of our gaming environment

In [ ]:
# Using pygame to create a window and initialiize it
s_width = 800
s_height = 800

win = pygame.display.set_mode((s_width, s_height))
pygame.display.init()

# Setting the caption of our game
pygame.display.set_caption('Tetris')

How do we render this? For the game window to continously appear on our screen and change over time, we need to display it using a loop. We will implement this loop inside a function called main:

In [ ]:
def main(win):

    while True:
        # Filling the window with black (0,0,0) pixels
        win.fill((0, 0, 0))

        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                # run = False
                break;

    pygame.quit()

Next, let's call this function to display our game window

In [ ]:
main(win)

You should see a black window:
<img src=placeholder_image.png width="200">


In a next step, let's create our grid

In [ ]:
# Variables to define the dimensions of our game display
s_width = 800
s_height = 700
play_width = 300  # meaning 300 // 10 = 30 width per block
play_height = 600  # meaning 600 // 20 = 20 height per block
block_size = 30

top_left_x = (s_width - play_width) // 2
top_left_y = s_height - play_height

We now need two functions to draw our window and our grid on top of it

In [ ]:
def draw_grid(surface, row, col):
    sx = top_left_x
    sy = top_left_y
    for i in range(row):
        pygame.draw.line(surface, (128, 128, 128), (sx, sy + i * 30),
                         (sx + play_width, sy + i * 30))  # horizontal lines
        for j in range(col):
            pygame.draw.line(surface, (128, 128, 128), (sx + j * 30, sy),
                             (sx + j * 30, sy + play_height))  # vertical lines

In [ ]:
def draw_window(surface):
    surface.fill((0, 0, 0))
    # Tetris Title
    font = pygame.font.SysFont('comicsans ', 60)
    label = font.render('TETRIS ', 1, (255, 255, 255))

    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), 30))

    for i in range(len(grid)):
        for j in range(len(grid[i])):
            pygame.draw.rect(surface, grid[i][j], (top_left_x + j * 30, top_left_y + i * 30, 30, 30), 0)

    # draw grid and border
    draw_grid(surface, 20, 10)
    pygame.draw.rect(surface, (255, 0, 0), (top_left_x, top_left_y, play_width, play_height), 5)

In [ ]:
def main_menu(win):

    while True:
        win.fill((0, 0, 0))
        draw_window(win)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                # run = False
                break;

    pygame.quit()

In [ ]:
# Let's test our window

main_menu(win)

You should see something like this:
<img src=placeholder_image.png width="200">

Now we have a window which displays our grid until we quit it. Next, we need to write our game

In [ ]:
# Let's define the datastructure which holds our shapes

# SHAPE FORMATS

S = [['.....',
    '......',
    '..00..',
    '.00...',
    '.....'],
    ['.....',
    '..0..',
    '..00.',
    '...0.',
    '.....']]

Z = [['.....',
'..... ',
 '.00.. ',
 '..00. ',
 '..... '],
[ '..... ',
 '..0.. ',
 '.00.. ',
 '.0... ',
 '..... ']]

I = [[ '..0.. ',
 '..0.. ',
 '..0.. ',
 '..0.. ',
 '..... '],
[ '..... ',
 '0000. ',
 '..... ',
 '..... ',
 '..... ']]

O = [[ '..... ',
 '..... ',
 '.00.. ',
 '.00.. ',
 '..... ']]

J = [[ '..... ',
 '.0... ',
 '.000. ',
 '..... ',
 '..... '],
[ '..... ',
 '..00. ',
 '..0.. ',
 '..0.. ',
 '..... '],
[ '..... ',
 '..... ',
 '.000. ',
 '...0. ',
 '..... '],
[ '..... ',
 '..0.. ',
 '..0.. ',
 '.00.. ',
 '..... ']]

L = [[ '..... ',
 '...0. ',
 '.000. ',
 '..... ',
 '..... '],
[ '..... ',
 '..0.. ',
 '..0.. ',
 '..00. ',
 '..... '],
[ '..... ',
 '..... ',
 '.000. ',
 '.0... ',
 '..... '],
[ '..... ',
 '.00.. ',
 '..0.. ',
 '..0.. ',
 '..... ']]

T = [[ '..... ',
 '..0.. ',
 '.000. ',
 '..... ',
 '..... '],
[ '..... ',
 '..0.. ',
 '..00. ',
 '..0.. ',
 '..... '],
[ '..... ',
 '..... ',
 '.000. ',
 '..0.. ',
 '..... '],
[ '..... ',
 '..0.. ',
 '.00.. ',
 '..0.. ',
 '..... ']]

shapes = [S, Z, I, O, J, L, T]
shape_colors = [(0, 255, 0), (255, 0, 0), (0, 255, 255), (255, 255, 0), (255, 165, 0), (0, 0, 255), (128, 0, 128)]

The shapes will look like this:
<img src=placeholder_image.png width="200">

Next, we need to creating the piece class. This is an object which will hold information about every piece in our game. You can think of it like a template which defines the properties such as teh position (x,y) and the color and shape of every piece

In [ ]:
class Piece(object):
    rows = 20  # y
    columns = 10  # x

    def __init__(self, column, row, shape):
        self.x = column
        self.y = row
        self.shape = shape
        self.color = shape_colors[shapes.index(shape)]
        self.rotation = 0  # number from 0-3

In a next step, we need to add the main function "play_tetris" to the window rendering function. This function will be the place where we define the rules of our game

In [ ]:
def draw_text_middle(text, size, color, surface):
    font = pygame.font.SysFont('comicsans', size, bold=True)
    label = font.render(text, 1, color)

    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2),
                         top_left_y + play_height / 2 - label.get_height() / 2))

def main(win):

    while True:
        win.fill((0, 0, 0))
        #draw_text_middle('Press any key to begin. ', 60, (255, 255, 255), win)
        pygame.display.update()
        play_tetris(win)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                # run = False
                break;
            
            # This is where we start our game!
            if event.type == pygame.KEYDOWN:
                play_tetris()
                
    pygame.quit()

What do we need to play our game?

First of all we need pieces that appear on the grid and fall down

The effect of falling down is achieved by creating a new grid over and over again (in a loop) with the pieces displaced at different positions. For this, we will define a function called 
"draw_grid", that takes as an input the occupied positions which it will draw on the empty black tetris background

In [ ]:
def create_grid(occupied_positions={}):
    grid = [[(0, 0, 0) for x in range(10)] for x in range(20)]

    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (j, i) in occupied_positions:
                c = occupied_positions[(j, i)]
                grid[i][j] = c
    return grid

Next, we need a function which gives us a random shape

In [ ]:
def get_shape():
    global shapes, shape_colors
    return Piece(5, 0, random.choice(shapes))

We will call this function every time we need a new shape

We also need a function which places the shape on the grid

In [ ]:
def place_shape_on_grid(shape):
    positions = []
    format = shape.shape[shape.rotation % len(shape.shape)]

    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column ==  '0':
                positions.append((shape.x + j, shape.y + i))


    for i, pos in enumerate(positions):
        positions[i] = (pos[0] - 2, pos[1] - 4)

    return positions

On to our main play_tetris function!

In [ ]:
def play_tetris(win):
    
    global grid

    occupied_positions = {}
    grid = create_grid(occupied_positions)

    run = True
    current_piece = get_shape()
    next_piece = get_shape()
    
    while run:
        
        shape_pos = place_shape_on_grid(current_piece)
        # add piece to the grid for drawing
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.color

        draw_window(win)
        pygame.display.update()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                # run = False
                break;


In [ ]:
main_menu(win)

Now you should see the following window
<img src=placeholder_image.png width="200">

Homework: 


1) Write some code inside the "play_tetris" function which makes this shape move down the grid. You will need to change the shapes' y position that you can access via shape.y and increase it (shape.y += 1) inside the loop. Make sure that you only increase the position every 20 iterations or so (use a counter), otherwise, your shape will move too quickly


Hint: 
create a counter variable counter = 0 and increase it inside the loop
counter += 1


Once the counter value has reached a specific value (for example 30 or 50)
increase the shape's position

if counter == 30:
    # increase the shapes y position
    

don't forget to reset the counter!

2) How would you define the shape's velocity down the grid if every loop iteration takes 2 nano seconds and you move down the shape every 20 iterations?


Moving the shape down the grid

In [ ]:
def play_tetris(win):
    
    global grid

    occupied_positions = {}
    grid = create_grid(occupied_positions)

    run = True
    current_piece = get_shape()
    next_piece = get_shape()
    
    while run:
        
        counter += 1
        # Move Shape down the grid
        if counter > threshold:
            current_piece.y += 1
            counter = 0
        
        #Don't forget to update the shape's position
        shape_pos = place_shape_on_grid(current_piece)
        # add piece to the grid for drawing
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.color

        draw_window(win)
        pygame.display.update()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                # run = False
                break;

You will now see shapes moving down the screen, but: They don't stop at the bottom! We need to ensure the piece does not leave the board. We will do this using the following function


In [ ]:
def valid_space(shape, grid):
    accepted_positions = [[(j, i) for j in range(10) if grid[i][j] == (0, 0, 0)] for i in range(20)]
    accepted_positions = [j for sub in accepted_positions for j in sub]
    formatted = convert_shape_format(shape)

    for pos in formatted:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False

    return True

Now we can use this function to ensure that our piece doesn't leave the bottom

In [ ]:
def play_tetris(win):
    
    global grid

    occupied_positions = {}
    grid = create_grid(occupied_positions)

    run = True
    current_piece = get_shape()
    next_piece = get_shape()
    
    counter = 0
    threshold = 5
    moving_shape_down = False
    
    change_piece = False
    
    while run:
        
        grid = create_grid(occupied_positions)
        
        counter += 1
        # Move Shape down the grid
        if counter > threshold:
            current_piece.y += 1
            counter = 0
             # moving_shape_down = False
        
        # check that if we have hit the ground
        if moving_shape_down:
            if not (valid_space(current_piece, grid)) and current_piece.y > 0:
                current_piece.y -= 1
                # when a piece hits the bottom we need to change piece to a new piece
                change_piece = True
      
       
        
        shape_pos = place_shape_on_grid(current_piece)
        # add piece to the grid for drawing
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.color
                
                
        

        draw_window(win)
        pygame.display.update()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                # run = False
                break;# IF PIECE HIT GROUND
        

We now have one piece moving down, and staying there. But how do we get another piece? We define a variable <b>change_piece</b> and use it to select another piece using our get_shape function

In [ ]:
def play_tetris(win):
    
    global grid

    occupied_positions = {}
    grid = create_grid(occupied_positions)

    run = True
    current_piece = get_shape()
    next_piece = get_shape()
    
    counter = 0
    threshold = 5
    moving_shape_down = False
    
    change_piece = False
    
    while run:
        
        grid = create_grid(occupied_positions)
        
        counter += 1
        # Move Shape down the grid
        if counter > threshold:
            current_piece.y += 1
            counter = 0
             # moving_shape_down = False
        
        # check that if we have hit the ground
        if moving_shape_down:
            if not (valid_space(current_piece, grid)) and current_piece.y > 0:
                current_piece.y -= 1
                # when a piece hits the bottom we need to change piece to a new piece
                change_piece = True
      
       
        
        shape_pos = place_shape_on_grid(current_piece)
        # add piece to the grid for drawing
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.color
                
        if change_piece:
            print("changing piece")
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_piece.color
                
            # update the piece
            current_piece = next_piece
            next_piece = get_shape()
            
            # set change piece to False
            change_piece = False
        

        draw_window(win)
        pygame.display.update()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                # run = False
                break;# IF PIECE HIT GROUND
        

We now have a nearly functioning Tetris game with shapes moving down the grid and stopping at the bottom. 
Next, we need to clear the rows when we completed them and log our score.
Let's start with clearing the rows.

In [ ]:
def clear_rows(grid, locked):
    # need to see if row is clear the shift every other row above down one

    inc = 0
    for i in range(len(grid) - 1, -1, -1):
        row = grid[i]
        if (0, 0, 0) not in row:
            inc += 1
            # add positions to remove from locked
            ind = i
            for j in range(len(row)):
                try:
                    del locked[(j, i)]
                except:
                    continue
    if inc > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                newKey = (x, y + inc)
                locked[newKey] = locked.pop(key)

Where do we call this function? 

In [ ]:
def play_tetris(win):
    
    global grid

    occupied_positions = {}
    grid = create_grid(occupied_positions)

    run = True
    current_piece = get_shape()
    next_piece = get_shape()
    
    counter = 0
    threshold = 5
    moving_shape_down = False
    
    change_piece = False
    
    while run:
        
        grid = create_grid(occupied_positions)
        
        counter += 1
        # Move Shape down the grid
        if counter > threshold:
            current_piece.y += 1
            counter = 0
             # moving_shape_down = False
        
        # check that if we have hit the ground
        if moving_shape_down:
            if not (valid_space(current_piece, grid)) and current_piece.y > 0:
                current_piece.y -= 1
                # when a piece hits the bottom we need to change piece to a new piece
                change_piece = True
        
        # Don't forget to update the shape's position
       
        
        shape_pos = place_shape_on_grid(current_piece)
        # add piece to the grid for drawing
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.color
                
                
        if change_piece:
            print("changing piece")
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_piece.color
                
            # update the piece
            current_piece = next_piece
            next_piece = get_shape()
            
            # set change piece to False
            change_piece = False

            # clear rows if a row is full
            clear_rows(grid, locked_positions)

        draw_window(win)
        pygame.display.update()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                # run = False
                break;# IF PIECE HIT GROUND

How could we use this function to log our score? We can modify the clear rows function to log the number of rows it clears and return the sum

In [ ]:
def clear_rows(grid, locked):
    # need to see if row is clear the shift every other row above down one

    inc = 0
    number_of_rows_cleared = 0
    for i in range(len(grid) - 1, -1, -1):
        row = grid[i]
        if (0, 0, 0) not in row:
            inc += 1
            number_of_rows_cleared +=1
            # add positions to remove from locked
            ind = i
            for j in range(len(row)):
                try:
                    del locked[(j, i)]
                except:
                    continue
    if inc > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                newKey = (x, y + inc)
                locked[newKey] = locked.pop(key)
                
    return number_of_rows_cleared

And in our main code base, we can now log the score using the return from our clear_rows function as follows:

In [ ]:
def play_tetris(win):
    global grid

    locked_positions = {}
    grid = create_grid(locked_positions)

    current_piece = get_shape()
    next_piece = get_shape()

    Run = True
    counter = 0
    delay = 10
    change_piece = False
    score = 0
    level_threshold = 1

    while (Run == True):

        grid = create_grid(locked_positions)

        counter += 1

        if score > level_threshold:
            delay -= 2
            level_threshold += 2


        if counter == delay:
            current_piece.y += 1
            counter = 0
            # the following code is to prevent the piece from going off the grid
            # valid_space = if the shape is on the grid
            if not valid_space(current_piece, grid) and current_piece.y > 0:
                current_piece.y -= 1
                change_piece = True


        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                Run = False
                pygame.display.quit()
                quit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_piece.x -= 1
                    if not valid_space(current_piece, grid):
                        current_piece.x += 1

                elif event.key == pygame.K_RIGHT:
                    current_piece.x += 1
                    if not valid_space(current_piece, grid):
                        current_piece.x -= 1
                elif event.key == pygame.K_UP:
                    # rotate shape
                    current_piece.rotation = current_piece.rotation + 1 % len(current_piece.shape)
                    if not valid_space(current_piece, grid):
                        current_piece.rotation = current_piece.rotation - 1 % len(current_piece.shape)

                if event.key == pygame.K_DOWN:
                    # move shape down
                    current_piece.y += 1
                    if not valid_space(current_piece, grid):
                        current_piece.y -= 1


        grid_position = convert_shape_format(current_piece)
        grid = draw_piece_on_grid(grid_position, current_piece, grid)


        if change_piece:
            print("changing piece")
            for pos in grid_position:
                p = (pos[0], pos[1])
                locked_positions[p] = current_piece.color

            score += clear_rows(grid, locked_positions)

            current_piece = next_piece
            next_piece = get_shape()

            change_piece = False

        if check_lost(locked_positions) == True:
            Run = False


        draw_window(win)
        pygame.display.update()

    win.fill((0, 0, 0))
    draw_text_middle("You lost! Failiure!", 50, (255, 255, 255), win)
    pygame.display.update()
    time.sleep(5)

Let's draw the score onto our game display

In [ ]:
def draw_text_score (surface, score):
    font = pygame.font.SysFont('timesnewroman', 30, bold=False)
    label = font.render('score', 1, (255, 255, 255))

    sx = top_left_x + play_width + 50
    sy = top_left_y + play_height / 2 - 100

    surface.blit(label, (sx + 10, sy - 30))

    score_value = font.render(str(score), 1, (255, 255, 255))

    surface.blit(score_value, (sx + 10, sy - 10))

In [ ]:
def play_tetris(win):
    global grid

    locked_positions = {}
    grid = create_grid(locked_positions)

    current_piece = get_shape()
    next_piece = get_shape()

    Run = True
    counter = 0
    delay = 10
    change_piece = False
    score = 0
    level_threshold = 1

    while (Run == True):

        grid = create_grid(locked_positions)

        counter += 1

        if score > level_threshold:
            delay -= 2
            level_threshold += 2


        if counter == delay:
            current_piece.y += 1
            counter = 0
            # the following code is to prevent the piece from going off the grid
            # valid_space = if the shape is on the grid
            if not valid_space(current_piece, grid) and current_piece.y > 0:
                current_piece.y -= 1
                change_piece = True


        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                Run = False
                pygame.display.quit()
                quit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_piece.x -= 1
                    if not valid_space(current_piece, grid):
                        current_piece.x += 1

                elif event.key == pygame.K_RIGHT:
                    current_piece.x += 1
                    if not valid_space(current_piece, grid):
                        current_piece.x -= 1
                elif event.key == pygame.K_UP:
                    # rotate shape
                    current_piece.rotation = current_piece.rotation + 1 % len(current_piece.shape)
                    if not valid_space(current_piece, grid):
                        current_piece.rotation = current_piece.rotation - 1 % len(current_piece.shape)

                if event.key == pygame.K_DOWN:
                    # move shape down
                    current_piece.y += 1
                    if not valid_space(current_piece, grid):
                        current_piece.y -= 1


        grid_position = convert_shape_format(current_piece)
        grid = draw_piece_on_grid(grid_position, current_piece, grid)


        if change_piece:
            print("changing piece")
            for pos in grid_position:
                p = (pos[0], pos[1])
                locked_positions[p] = current_piece.color

            score += clear_rows(grid, locked_positions)

            current_piece = next_piece
            next_piece = get_shape()

            change_piece = False

        if check_lost(locked_positions) == True:
            Run = False


        draw_window(win)
        draw_text_score(win, score)
        pygame.display.update()

    win.fill((0, 0, 0))
    draw_text_middle("You lost! Failiure!", 50, (255, 255, 255), win)
    pygame.display.update()
    time.sleep(5)

To help with playing the game, let's draw the next upcoming shape next to our grid

Finally, let's add levels to the game